## Fine tuning models

In [1]:
import torch
import torch.nn as nn

from torch.autograd.variable import Variable
from torchvision import datasets, transforms, models

In [2]:
model = models.resnet18(pretrained=False)

In [3]:
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Co

### Freezing the layers of Resnet18 for fine-tuning

In [6]:
child_counter = 0
for child in model.children():
    print('| => child', child_counter, 'is -')
    print(child)
    child_counter += 1

| => child 0 is -
Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
| => child 1 is -
BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
| => child 2 is -
ReLU(inplace)
| => child 3 is -
MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
| => child 4 is -
Sequential(
  (0): BasicBlock(
    (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (1): BasicBlock(
    (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace)
    (conv2):

In [7]:
for child in model.children():
    for param in child.parameters():
        print('| => This is what a parameter look like - \n', param)
        break
    break

| => This is what a parameter look like - 
 Parameter containing:
tensor([[[[ 0.0088, -0.0333,  0.0082,  ..., -0.0132, -0.0288,  0.0316],
          [-0.0093,  0.0176, -0.0167,  ..., -0.0090, -0.0180, -0.0033],
          [-0.0186,  0.0516,  0.0409,  ..., -0.0648,  0.0280, -0.0465],
          ...,
          [ 0.0059, -0.0351, -0.0129,  ..., -0.0207,  0.0098, -0.0245],
          [ 0.0158,  0.0078,  0.0070,  ...,  0.0201,  0.0307,  0.0301],
          [-0.0335, -0.0069, -0.0032,  ..., -0.0267,  0.0230,  0.0488]],

         [[ 0.0073, -0.0234,  0.0013,  ...,  0.0018, -0.0080,  0.0140],
          [ 0.0040, -0.0073, -0.0655,  ..., -0.0265, -0.0607,  0.0064],
          [ 0.0072,  0.0000, -0.0265,  ...,  0.0420, -0.0348, -0.0172],
          ...,
          [ 0.0387, -0.0155,  0.0047,  ...,  0.0177, -0.0401,  0.0145],
          [-0.0052, -0.0079, -0.0327,  ...,  0.0448, -0.0537, -0.0195],
          [-0.0281,  0.0037, -0.0233,  ..., -0.0188,  0.0093,  0.0092]],

         [[ 0.0008,  0.0243, -0.0341

##### Freeze up to first basicblock of child 6

In [9]:
child_counter = 0
for child in model.children():
    if child_counter < 6:
        print('| Child ', child_counter, ' was frozen')
        for param in child.parameters():
            param.requires_grad = False
            
    elif child_counter == 6:
        child_of_child_counter = 0
        for children_of_child in child.children():
            if child_of_child_counter < 1:
                for param in children_of_child.parameters():
                    param.requires_grad = False
                print('| Child ', child_of_child_counter, ' of child ', child_counter, ' was frozen')
            else:
                print('| Child ', child_of_child_counter, ' of child ', child_counter, ' was not frozen')
            child_of_child_counter += 1
        
    else:
        print('| Child ', child_counter, ' was not frozen')
    child_counter += 1

| Child  0  was frozen
| Child  1  was frozen
| Child  2  was frozen
| Child  3  was frozen
| Child  4  was frozen
| Child  5  was frozen
| Child  0  of child  6  was frozen
| Child  1  of child  6  was not frozen
| Child  7  was not frozen
| Child  8  was not frozen
| Child  9  was not frozen


## Model Saving/Loading

### Changing last layer, deleting last layer, adding layers